In [71]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate

In [72]:
# Reading ratings file
movies = pd.read_csv('/Users/eeshnaugraiya/Downloads/ml-100k/u.item', sep='|', encoding='latin-1', header=None)

# Select the desired columns by index
selected_columns = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

# Assign the selected columns to the DataFrame and set column names
movies = movies.iloc[:, selected_columns]

# Set column names
movies.columns = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
                  'unknown', 'Action', 'Adventure', 'Animation', 'Childrens',
                  'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                  'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
                  'Sci-Fi', 'Thriller', 'War', 'Western']

# Now you have the desired columns in the 'movies' DataFrame



# Reading users file
# Reading ratings file
users = pd.read_csv('/Users/eeshnaugraiya/Downloads/ml-100k/u.user', sep='|', encoding='latin-1', header=None)
selected_columns = [0, 1, 2, 3, 4]

# Assign the selected columns to the DataFrame and set column names
users = users.iloc[:, selected_columns]

# Set column names
users.columns = ['user_id', 'age', 'gender', 'occupation' , 'zip_code']


# Reading ratings file

ratings = pd.read_csv('/Users/eeshnaugraiya/Downloads/ml-100k/u.data', sep='\t', encoding='latin-1', header=None)
selected_columns = [0, 1, 2, 3]

ratings = ratings.iloc[:, selected_columns]

# Set column names
ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

In [73]:
movies.head()



,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [74]:
users.head()


,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [75]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [76]:
# Fill missing values in the 'movie_title' column with empty strings
movies['movie_title'] = movies['movie_title'].fillna('')

# Display the first few rows of the 'movie_title' column after filling missing values
print(movies['movie_title'].head())



0     Toy Story (1995)
1     GoldenEye (1995)
2    Four Rooms (1995)
3    Get Shorty (1995)
4       Copycat (1995)
Name: movie_title, dtype: object


In [77]:
# Check the datatype of "id" in movies_metadata.csv
pd.DataFrame({'feature':movies.dtypes.index, 'dtype':movies.dtypes.values})

,feature,dtype
0,movie_id,int64
1,movie_title,object
2,release_date,object
3,video_release_date,float64
4,IMDb_URL,object
5,unknown,int64
6,Action,int64
7,Adventure,int64
8,Animation,int64
9,Childrens,int64


In [78]:
import pandas as pd
import numpy as np

# Load your movie data
# Assuming your DataFrame is named 'movies_df'
# Replace this with your actual data loading code
#movies_df = pd.read_csv('/Users/eeshnaugraiya/Downloads/ml-100k/u.item', delimiter='|')

# Define a function to get movie recommendations based on genres
def get_genre_recommendations(movie_id, top_n=10):
    # Extract the genre columns from the DataFrame
    genre_columns = ['unknown', 'Action', 'Adventure', 'Animation', 'Childrens',
                     'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
                     'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

    # Get the genre values for the given movie
    movie_genres = movies.iloc[movie_id][genre_columns]

    # Find movies with similar genres
    similar_movies = []
    for idx, row in movies.iterrows():
        if idx == movie_id:
            continue

        # Calculate the Jaccard similarity between genres
        intersection = np.sum(movie_genres & row[genre_columns])
        union = np.sum(movie_genres | row[genre_columns])
        jaccard_similarity = intersection / union

        similar_movies.append((idx, jaccard_similarity))

    # Sort the similar movies by Jaccard similarity
    similar_movies.sort(key=lambda x: x[1], reverse=True)

    # Get the top N similar movies
    top_similar_movies = similar_movies[:top_n]

    # Get the movie titles for the top similar movies
    recommended_movie_ids = [movie[0] for movie in top_similar_movies]
    recommended_movies = movies.loc[recommended_movie_ids]['movie_title']

    return recommended_movies

# Example: Get genre-based recommendations for a movie (replace 0 with the actual movie ID)
movie_id_to_recommend_for = 1
genre_based_recommendations = get_genre_recommendations(movie_id_to_recommend_for)
print(genre_based_recommendations)


116                    Rock, The (1996)
117                      Twister (1996)
565     Clear and Present Danger (1994)
826                     Daylight (1996)
929               Chain Reaction (1996)
981                 Maximum Risk (1996)
1012                    Anaconda (1997)
1015                     Con Air (1997)
1104                   Firestorm (1998)
1313          Surviving the Game (1994)
Name: movie_title, dtype: object


In [80]:
def get_genre_recommendations(movie_title, top_n=10):
    # Extract the genre columns from the DataFrame
    genre_columns = ['unknown', 'Action', 'Adventure', 'Animation', 'Childrens',
                     'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
                     'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

    # Find the movie with the given title
    movie = movies[movies['movie_title'] == movie_title]

    if movie.empty:
        return []

    # Get the genre values for the given movie
    movie_genres = movie[genre_columns].values[0]

    # Find movies with similar genres
    similar_movies = []
    for idx, row in movies.iterrows():
        if row['movie_title'] == movie_title:
            continue

        # Calculate the Jaccard similarity between genres
        intersection = np.sum(movie_genres & row[genre_columns])
        union = np.sum(movie_genres | row[genre_columns])
        jaccard_similarity = intersection / union

        similar_movies.append((row['movie_title'], jaccard_similarity))

    # Sort the similar movies by Jaccard similarity
    similar_movies.sort(key=lambda x: x[1], reverse=True)

    # Get the top N similar movies
    top_similar_movies = similar_movies[:top_n]

    return top_similar_movies

# Example: Get genre-based recommendations for a movie title (replace with your movie title)
movie_title_to_recommend_for = 'Terminator 2: Judgment Day (1991)'
genre_based_recommendations = get_genre_recommendations(movie_title_to_recommend_for)
print(genre_based_recommendations)
for recommendation in genre_based_recommendations:
    print(recommendation[0])

[('Lawnmower Man, The (1992)', 1.0), ('Terminator, The (1984)', 1.0), ('Face/Off (1997)', 1.0), ('Johnny Mnemonic (1995)', 1.0), ('Arrival, The (1996)', 1.0), ('Lost in Space (1998)', 1.0), ('Solo (1996)', 1.0), ('Nemesis 2: Nebula (1995)', 1.0), ('Abyss, The (1989)', 0.75), ('Aliens (1986)', 0.75)]
Lawnmower Man, The (1992)
Terminator, The (1984)
Face/Off (1997)
Johnny Mnemonic (1995)
Arrival, The (1996)
Lost in Space (1998)
Solo (1996)
Nemesis 2: Nebula (1995)
Abyss, The (1989)
Aliens (1986)


In [ ]:
#user based collaborative filtering

In [89]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Create a Reader object for Surprise
reader = Reader(rating_scale=(1, 5))

# Load the dataset
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.25)

# Create and train the SVD model
svd_model = SVD()
svd_model.fit(trainset)

# Make predictions for the testset
user_predictions = svd_model.test(testset)

# Recommend top N movies for the user
def get_top_n(predictions, n=10):
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n_recommendations = get_top_n(user_predictions, n=10)


In [99]:
# Replace user_id_to_recommend_for with the desired user_id
user_id_to_recommend_for = 1

# Get the list of all movie IDs
movie_ids = ratings['movie_id'].unique()

# Create a list of movie IDs that the user has already rated
movies_rated_by_user = ratings[ratings['user_id'] == user_id_to_recommend_for]['movie_id'].tolist()

# Filter out movies that the user has already rated
movies_to_predict = [movie_id for movie_id in movie_ids if movie_id not in movies_rated_by_user]

# Predict ratings for the remaining movies
user_predictions = [svd_model.predict(user_id_to_recommend_for, movie_id) for movie_id in movies_to_predict]

# Sort the predictions by estimated rating in descending order
user_predictions.sort(key=lambda x: x.est, reverse=True)

# Get the top N movie recommendations (e.g., top 10)
top_n_recommendations = user_predictions[:10]

# Extract movie IDs from the recommendations
recommended_movie_ids = [prediction.iid for prediction in top_n_recommendations]

# Get the movie titles for the recommended movie IDs
recommended_movies = movies[movies['movie_id'].isin(recommended_movie_ids)]['movie_title']

# Print the recommended movies
print(recommended_movies)


275                  Leaving Las Vegas (1995)
301                  L.A. Confidential (1997)
317                   Schindler's List (1993)
356    One Flew Over the Cuckoo's Nest (1975)
407                     Close Shave, A (1995)
483                Maltese Falcon, The (1941)
510                 Lawrence of Arabia (1962)
526                             Gandhi (1982)
653                          Chinatown (1974)
656          Manchurian Candidate, The (1962)
Name: movie_title, dtype: object


In [97]:
# Replace user_id_to_recommend_for with the desired user_id
user_id_to_recommend_for = 1

# Replace last_movie_seen with the title of the last movie seen by the user
last_movie_seen = 'Terminator 2: Judgment Day (1991)'

# Get the movie ID for the last movie seen by the user
last_movie_id = movies[movies['movie_title'] == last_movie_seen]['movie_id'].values[0]

# Get the list of all movie IDs
movie_ids = ratings['movie_id'].unique()

# Create a list of movie IDs that the user has already rated
movies_rated_by_user = ratings[ratings['user_id'] == user_id_to_recommend_for]['movie_id'].tolist()

# Filter out the last movie seen (if it exists)
if last_movie_id in movies_rated_by_user:
    movies_rated_by_user.remove(last_movie_id)

# Filter out movies that the user has already rated (including the last movie seen)
movies_to_predict = [movie_id for movie_id in movie_ids if movie_id not in movies_rated_by_user]

# Predict ratings for the remaining movies
user_predictions = [svd_model.predict(user_id_to_recommend_for, movie_id) for movie_id in movies_to_predict]

# Sort the predictions by estimated rating in descending order
user_predictions.sort(key=lambda x: x.est, reverse=True)

# Get the top N movie recommendations (e.g., top 10)
top_n_recommendations = user_predictions[:10]

# Extract movie IDs from the recommendations
recommended_movie_ids = [prediction.iid for prediction in top_n_recommendations]

# Get the movie titles for the recommended movie IDs
recommended_movies = movies[movies['movie_id'].isin(recommended_movie_ids)]['movie_title']

# Print the recommended movies
print(recommended_movies)


275                  Leaving Las Vegas (1995)
301                  L.A. Confidential (1997)
317                   Schindler's List (1993)
356    One Flew Over the Cuckoo's Nest (1975)
407                     Close Shave, A (1995)
483                Maltese Falcon, The (1941)
510                 Lawrence of Arabia (1962)
526                             Gandhi (1982)
653                          Chinatown (1974)
656          Manchurian Candidate, The (1962)
Name: movie_title, dtype: object


In [92]:
def hybrid_recommendations(user_id, movie_title, top_n=10, cf_weight=0.7, cb_weight=0.3):
    # Get user-based CF recommendations
    cf_recommendations = top_n_recommendations.get(user_id, [])

    # Get content-based recommendations
    cb_recommendations = get_genre_recommendations(movie_title, top_n)

    # Combine recommendations using weighted average
    hybrid_recommendations = {}
    for movie_id, cf_score in cf_recommendations:
        movie_title = movies[movies['movie_id'] == movie_id]['movie_title'].values[0]
        hybrid_recommendations[movie_title] = cf_weight * cf_score

    for movie_title, cb_score in cb_recommendations:
        if movie_title in hybrid_recommendations:
            hybrid_recommendations[movie_title] += cb_weight * cb_score
        else:
            hybrid_recommendations[movie_title] = cb_weight * cb_score

    # Sort the hybrid recommendations by score
    sorted_hybrid_recommendations = sorted(hybrid_recommendations.items(), key=lambda x: x[1], reverse=True)

    # Get the top N hybrid recommendations
    top_hybrid_recommendations = sorted_hybrid_recommendations[:top_n]

    return top_hybrid_recommendations


In [94]:
user_id_to_recommend_for = 1  # Replace with the user_id
last_movie_seen = 'Terminator 2: Judgment Day (1991)'  # Replace with the movie title

hybrid_rec = hybrid_recommendations(user_id_to_recommend_for, last_movie_seen)
for movie_title, score in hybrid_rec:
    print(movie_title)


Sling Blade (1996)
Good, The Bad and The Ugly, The (1966)
Empire Strikes Back, The (1980)
This Is Spinal Tap (1984)
Cinema Paradiso (1988)
Fish Called Wanda, A (1988)
Hoop Dreams (1994)
Brazil (1985)
Priest (1994)
Bound (1996)


In [34]:
#user-based collaborative filtering

In [81]:
# Create a user-item matrix
user_item_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')


In [82]:
# Load data into Surprise Dataset
reader = Reader()
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Build and train the SVD model
svd = SVD()
trainset = data.build_full_trainset()
svd.fit(trainset)


In [83]:
def get_user_based_recommendations(user_id, top_n=10):
    # Predict ratings for all movies for the given user
    predictions = []
    for movie_id in user_item_matrix.columns:
        if pd.isnull(user_item_matrix.loc[user_id, movie_id]):
            prediction = svd.predict(user_id, movie_id)
            predictions.append((movie_id, prediction.est))

    # Sort the predictions by estimated rating in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Get the top N recommended movie IDs
    top_movie_ids = [movie_id for movie_id, _ in predictions[:top_n]]

    # Get movie titles for the top recommended movies
    recommended_movies = movies[movies['movie_id'].isin(top_movie_ids)]['movie_title']
    
    return recommended_movies


In [86]:
def get_hybrid_recommendations(user_id, top_n=10):
    # Get recommendations from content-based model
    content_based_recommendations = get_genre_recommendations(user_id, top_n=top_n)
    
    # Get recommendations from user-based model
    user_based_recommendations = get_user_based_recommendations(user_id, top_n=top_n)
    
    # Check if either of the recommendation lists is empty
    if not content_based_recommendations:
        return user_based_recommendations[:top_n]
    elif not user_based_recommendations:
        return content_based_recommendations[:top_n]
    
    # Combine the recommendations from both models
    hybrid_recommendations = content_based_recommendations + user_based_recommendations
    
    # Remove duplicates and keep the top N recommendations
    hybrid_recommendations = list(set(hybrid_recommendations))[:top_n]
    
    return hybrid_recommendations



In [87]:
user_id_to_recommend_for = 1  # Replace with the actual user ID
hybrid_recommendations = get_hybrid_recommendations(user_id_to_recommend_for)
print(hybrid_recommendations)


275                             Leaving Las Vegas (1995)
284                                Secrets & Lies (1996)
316                     In the Name of the Father (1993)
317                              Schindler's List (1993)
407                                Close Shave, A (1995)
473    Dr. Strangelove or: How I Learned to Stop Worr...
477                       Philadelphia Story, The (1940)
478                                       Vertigo (1958)
482                                    Casablanca (1942)
602                                   Rear Window (1954)
Name: movie_title, dtype: object


In [37]:
# -- User-focused recommender --

reader = Reader() # Used to parse a file containing ratings
df = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(df) # Split the data into folds

<generator object KFold.split at 0x177461300>

In [38]:
# Use Single Value Decomposition (SVD) for cross-validation and fitting
svd = SVD()
cross_validate(svd, df, measures=['RMSE', 'MAE'])

trainset = df.build_full_trainset()
svd.fit(trainset)

In [40]:
# Check a random user's ratings
ratings[ratings['user_id'] == 10]


,user_id,movie_id,rating,timestamp
40,10,16,4,877888877
158,10,486,4,877886846
386,10,175,3,877888677
544,10,611,5,877886722
606,10,7,4,877892210
...,...,...,...,...
94371,10,686,4,877886911
96580,10,604,4,877892110
97430,10,93,4,877892160
99578,10,9,4,877889005


In [47]:
movie_title_to_id = dict(zip(movies['movie_title'], movies['movie_id']))

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
movie_title_to_id = dict(zip(movies['movie_title'], movies['movie_id']))

# Merge ratings data with movie titles
ratings = ratings.merge(movies[['movie_id', 'movie_title']], on='movie_id')

# Create a user-item rating matrix
user_item_matrix = ratings.pivot_table(index='user_id', columns='movie_title', values='rating')

# Fill missing values with 0 (assuming no rating means a rating of 0)
user_item_matrix = user_item_matrix.fillna(0)

# Calculate the cosine similarity between movies based on user ratings
cosine_sim = cosine_similarity(user_item_matrix)



In [67]:
def hybrid(user_id, movie_title, cosine_sim, svd):
    # Get the movie_id of the selected movie title using the movie_title_to_id dictionary
    movie_id = movie_title_to_id.get(movie_title)
    
    # Calculate similarity scores based on user ratings
    sim_scores = list(enumerate(cosine_sim[movie_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]  # Top 30 most similar movies
    
    # Get the movie_ids of similar movies
    similar_movie_ids = [i[0] for i in sim_scores]
    
    # Get movie titles and predicted ratings using SVD
    similar_movies = movies[movies['movie_id'].isin(similar_movie_ids)]
    similar_movies['predicted_rating'] = similar_movies['movie_id'].apply(lambda x: svd.predict(user_id, x).est)
    
    # Sort the similar movies by predicted rating
    similar_movies = similar_movies.sort_values(by='predicted_rating', ascending=False)
    
    return similar_movies.head(15)

# Example usage of the hybrid recommender
user_id = 2
movie_title = "Toy Story (1995)"
recommendations = hybrid(user_id, movie_title, cosine_sim, svd)
print(recommendations[['movie_title', 'predicted_rating']])

                                           movie_title  predicted_rating
518           Treasure of the Sierra Madre, The (1948)          4.234964
461  Like Water For Chocolate (Como agua para choco...          4.196261
935                                 Brassed Off (1996)          4.149822
264                   Hunt for Red October, The (1990)          4.082576
524                              Big Sleep, The (1946)          4.066077
458                    Cry, the Beloved Country (1995)          4.038543
484                                My Fair Lady (1964)          3.929656
567                                       Speed (1994)          3.918453
275                           Leaving Las Vegas (1995)          3.823033
13                                  Postino, Il (1994)          3.820990
741                                      Ransom (1996)          3.762995
304                              Ice Storm, The (1997)          3.539457
294                                   Breakdown (19

/var/folders/wn/28xgtwmj751dk2ks6jz_6d840000gn/T/ipykernel_35972/4038544394.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_movies['predicted_rating'] = similar_movies['movie_id'].apply(lambda x: svd.predict(user_id, x).est)


In [ ]:
# Recommendation function
def hybrid(user_id, movie_title):
    idx = indices[movie_title]
   
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] # Scores of the 30 most similar movies
    movie_indices = [i[0] for i in sim_scores]
    
    movies = meta.iloc[movie_indices][['title', 'vote_average', 'tmdbId']]
    movies['est'] = movies['tmdbId'].apply(lambda x: svd.predict(userId, links_index.loc[x]['movieId']).est) # Estimated prediction using svd
    movies = movies.sort_values('est', ascending=False) # Rank movies according to the predicted values
    movies.columns = ['movie_itle', 'Vote Average', 'TMDb Id', 'Estimated Prediction']
    return movies.head(15) # Display top 15 similar movies
